<Strong> Develop a TCN model that overfits , then do some regularization , then do some tuning. </Strong>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop , Adam
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error , mean_absolute_error , mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM , Dense , Dropout , GRU , Concatenate , Input , Conv1D , InputLayer
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
import math

In [2]:
#df_btc = pd.read_csv("../../cryptoData/BTC_1h_data.csv")
df_btc = pd.read_csv("BTC_1h_data.csv")

<Strong> Grabbing the closing price (univariate) </Strong>

In [3]:
# Removing everything but the closing price
btc_data = df_btc.values[:, 4 ,].astype(float)

btc_data[-1]

43534.54

<Strong> Scaling the data  </Strong>

In [4]:
percTrain = 70
percVal = 20 

In [5]:
scaler = MinMaxScaler()
    
onePercent = len(btc_data) // 100
numberTraining = onePercent * percTrain

reshaped_data = btc_data.reshape(-1,1)

#Just scaling on training data otherwise it would be leakage
scaler.fit(reshaped_data[:numberTraining])
scaled_btc = scaler.transform(reshaped_data)

<Strong> Creating Matrix in Sliding window form <Strong>

In [6]:
def sliding_window(elements, window_size):
    
    data = [] 
    targets = []
    
    if len(elements) <= window_size:
        return elements
    
    for i in range(len(elements) - window_size ):
        
        data.append(elements[i:i+window_size])
        targets.append(elements[i+window_size])
        
    return np.array(data) , np.array(targets)

In [7]:
# Using 24 datapoints to predict the 25th

window_length = 24
features = 1

sliding_winda_btc = sliding_window(scaled_btc , window_length)

<Strong> Splitting the data into train , val , test </Strong>

In [8]:
# Splitting the data after creating the sliding window data
def splitting_train_test(data):
        
    onePercent = len(data[1]) // 100
    
    numberTraining = onePercent * percTrain
    numberValidation = onePercent * percVal
    
    trainingData = data[0][:numberTraining] , data[1][:numberTraining]
    validationData = data[0][numberTraining : numberTraining + numberValidation] , data[1][numberTraining : numberTraining + numberValidation]
    testData = data[0][numberTraining + numberValidation:] , data[1][numberTraining + numberValidation:] 
    
    #Returning tuples of (sliding-window , target_values)
    return trainingData , validationData , testData

In [9]:
btc_train , btc_val , btc_test = splitting_train_test(sliding_winda_btc)

print(btc_train[0].shape)

(27370, 24, 1)


In [45]:
# Hyper Parameters To Overfit 

# Add Layers 
# Train longer


In [17]:
# Full Converage = 1 + (kernel_size-1) . dilation_base^n - 1 / dilation_base - 1 >= l
# Number of required layers = log (dilation_base) ( ((input_length - 1) . (dilation_base - 1) / (k - 1) ) + 1 )
# Padding for each layer dilation_base ^ layers below our current . (kernel_size - 1) below = input

# For full coverage we need 4 layers (we are using a base of 2)


model1 = models.Sequential()

# Casual adds padding to the start of input sequence
model1.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(window_length, 1), dilation_rate=1 , padding = 'causal'))

model1.add(Conv1D(filters=64, kernel_size=3, activation='relu' , padding = 'causal', dilation_rate=2))

# For full coverage we need 4 layers 
model1.add(Conv1D(filters=128, kernel_size=3, activation='relu' , padding = 'causal' , dilation_rate=4))

# For full coverage we need 4 layers 
model1.add(Conv1D(filters=128, kernel_size=3, activation='relu' , padding = 'causal', dilation_rate= 8))

model1.add(Dense(128))

model1.add(Dense(1))


model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 24, 32)            128       
                                                                 
 conv1d_9 (Conv1D)           (None, 24, 64)            6208      
                                                                 
 conv1d_10 (Conv1D)          (None, 24, 128)           24704     
                                                                 
 conv1d_11 (Conv1D)          (None, 24, 128)           49280     
                                                                 
 dense_2 (Dense)             (None, 24, 128)           16512     
                                                                 
 dense_3 (Dense)             (None, 24, 1)             129       
                                                                 
Total params: 96,961
Trainable params: 96,961
Non-trai

In [18]:
opt = Adam(0.008)
model1.compile(optimizer=opt , loss = 'mse')

In [19]:
history = model1.fit(btc_train[0] , btc_train[1] , validation_data = btc_val  , batch_size = 512  , epochs =300 , verbose = 1 )

num_epochs = earlyStopping.stopped_epoch

Epoch 1/300
54/54 [==============================] - 2s 18ms/step - loss: 0.0311 - val_loss: 0.0722
Epoch 2/300
54/54 [==============================] - 1s 12ms/step - loss: 5.3316e-04 - val_loss: 0.0142
Epoch 3/300
54/54 [==============================] - 1s 12ms/step - loss: 4.1318e-04 - val_loss: 0.0090
Epoch 4/300
54/54 [==============================] - 1s 12ms/step - loss: 3.4430e-04 - val_loss: 0.0106
Epoch 5/300
54/54 [==============================] - 1s 12ms/step - loss: 3.3926e-04 - val_loss: 0.0073
Epoch 6/300
54/54 [==============================] - 1s 12ms/step - loss: 3.7281e-04 - val_loss: 0.0069
Epoch 7/300
54/54 [==============================] - 1s 12ms/step - loss: 3.3621e-04 - val_loss: 0.0100
Epoch 8/300
54/54 [==============================] - 1s 12ms/step - loss: 3.3572e-04 - val_loss: 0.0065
Epoch 9/300
54/54 [==============================] - 1s 12ms/step - loss: 3.2296e-04 - val_loss: 0.0092
Epoch 10/300
54/54 [==============================] - 1s 12ms/step -

KeyboardInterrupt: 